In [ ]:
## made .txt file of same name, gave me txt version of page with metadata, interesting!
# or not? ican open txt file of it either way

In [239]:
# imports
import pandas as pd
import numpy as np
import spotipy #pip install spotipy first
import os
from client import *
from spotipy.oauth2 import SpotifyClientCredentials # required to access user data
import spotipy.util as util

### Spotify Authorization

In [240]:
user= userID
secret = secret
cid = clientID

In [241]:
def get_token(scope=None): #get token and set scope later if want
    redirect_uri = "http://localhost:4000"
    token = util.prompt_for_user_token(user, scope, cid, secret, redirect_uri)
    return token

In [281]:
token = get_token(scope='playlist-modify-private') # can make public later

sp = spotipy.Spotify(auth=token)

### Initialize the Playlist and Main Dataframe

In [243]:
# run once, use playlist_id to ADD the songs to
def initialize_playlist():
    playlist = sp.user_playlist_create(user, 'AutoRepeatAll', public = False)
    playlist_id = playlist["id"]
    return playlist_id

In [244]:
AutoRepeatAll_id = initialize_playlist()

In [245]:
# playlist urls to get data from
playlist_urls =['37i9dQZF1EpoJIsGQLtjPv', '37i9dQZF1EppjlneYJWXxZ', '37i9dQZF1Epf11aYgU5Q4u', '37i9dQZF1EpoxPdVWcv3CP', '37i9dQZF1EpwZ0YRZwMO7Y', '37i9dQZF1EpoCGPknzdJwy', '37i9dQZF1EphQVRnk9uGI6', '37i9dQZF1Epg86odGM5hPi', '37i9dQZF1EpL8wYDpLK6E6', '37i9dQZF1EpfkWyPBlEQKr']
creator = "spotify"

In [246]:
# make df to hold items to be added to playlist?
columns = ["artist", "album", "track_name", "track_id"]
main_df = pd.DataFrame(columns = columns)

### Get the songs from the playlists

In [248]:
# gets playlist tracks from single playlist and makes a df with track data for single playlist

def playlist_to_df(creator, playlist_id):
    playlist_df = pd.DataFrame(columns=columns)
    playlist_items = sp.user_playlist_tracks(creator, playlist_id)["items"] # returns item detail for each song
    for track in playlist_items:
        track_features = {} # make dict for each track with appropriate info
        track_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        track_features["album"] = track["track"]["album"]["name"]
        track_features["track_name"] = track["track"]["name"]
        track_features["track_id"] = track["track"]["id"]
        
        track_df = pd.DataFrame(track_features, index=[0]) # make df from dict for each item
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True) # add it to the ma
        
    return playlist_df
#https://www.linkedin.com/pulse/extracting-your-fav-playlist-info-spotifys-api-samantha-jones/

In [249]:
# makes list of dfs for each playlist_id in playlist_urls list
# returns list of dfs

def make_playlist_df_list(playlist_urls):
    playlist_df_list = []
    for playlist in playlist_urls:
        playlist_df_list.append(playlist_to_df(creator, playlist))
    return playlist_df_list

In [251]:
df_playlist_list = make_playlist_df_list(playlist_urls)

In [260]:
# concats all track info into one main_df
# returns main_df, duplicates removed
def add_playlist_dfs_to_main_df():
    main_df = pd.concat(df_playlist_list, axis=0, ignore_index=True)
    main_df = main_df.drop_duplicates()
    return main_df

In [261]:
main_df = add_playlist_dfs_to_main_df()

### Make list of track_uris and split into batches

In [270]:
all_tracks = main_df["track_id"].tolist()

In [272]:
# split into batches:
# use yield for a generator function
def make_batches(l, n):
    for i in range(0, len(l), n): 
        yield l[i:i + n]

In [277]:
batches = list(make_batches(all_tracks, 99))

### Add tracks from main_df to AutoRepeatAll playlist

In [279]:
def add_tracks_to_main_playlist():
    for batch in batches:
        sp.playlist_add_items(playlist_id=AutoRepeatAll_id, items=batch)
    return print("confirm tracks added in spotify app")

In [282]:
add_tracks_to_main_playlist()

confirm tracks added in spotify app


In [115]:
track_list = df1["track_id"].tolist()

In [128]:
def make_playlist(df, track_list):
    playlist = sp.user_playlist_create(user, 'autoRepeat', public = False)
    playlist_id = playlist["id"]
    
    sp.user_playlist_add_tracks(user, playlist_id=playlist_id, tracks=track_list)
    return playlist_id

In [129]:
make_playlist(df1, track_list)

'7q1aLfR34kcOI3jhwMM1ah'